In [1]:
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap.foliumap as geemap
import sys

sys.path.append('../../')

from src.ee_scripts.ee_utils import *
from src.data.data_utils import *

ee.Authenticate()
ee.Initialize()

In [93]:
GAUL1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
GAUL0 = ee.FeatureCollection('FAO/GAUL/2015/level0')

germany = GAUL0.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
provinces = GAUL1.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
provinces = provinces.toList(provinces.size().getInfo())
subset1_geom = ee.FeatureCollection(provinces.slice(0, int(0.5 * provinces.size().getInfo())))
subset2_geom = ee.FeatureCollection(provinces.slice(int(0.5 * provinces.size().getInfo()), provinces.size().getInfo()))

country = GAUL0.filter(ee.Filter.eq('ADM0_NAME', 'Slovenia'))



In [94]:
Map = geemap.Map()
Map.addLayer(country)
Map.centerObject(country)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [95]:
roi = country
start_date = '2018-01-01'
end_date = '2019-01-01'
year = '2018'
step = 10 #Days to step for averages
export_scale = 10

lucas_subset =  (
    ee.FeatureCollection('projects/ee-ayang115t/assets/lucas_2018_filtered_polygons')
    .filterBounds(roi)
)
s1 = generate_s1_averages(start_date, end_date, roi, step)
histar = extract_histar(roi, start_date, end_date, step)

fused = s1.addBands(histar)

In [96]:
imageVisParam = {
    "opacity": 1,
    "bands": [
        "13_B3_mean_post_20180511",
        "13_B2_mean_post_20180511",
        "13_B1_mean_post_20180511"
    ],
    'min': 0,
    'max': 2500,
    'gamma': 1
}

s1VisParams = {
    "min": -25, 
    "max": 0
}

Map = geemap.Map()

Map.addLayer(fused.select("13_VV_20180511").clip(roi), s1VisParams, "VV")
Map.addLayer(fused.select("13_VH_20180511").clip(roi), s1VisParams, "VH")
Map.addLayer(fused.clip(roi), imageVisParam, "visual")
Map.addLayer(lucas_subset, {"color":"red"}, "Lucas points")

In [97]:
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [98]:
to_export = fused.sampleRegions(
    collection = lucas_subset,
    properties = ['POINT_ID','stratum'],
    tileScale = 16,
    scale = export_scale,
    geometries=True
)

fused_df = ee.data.computeFeatures({
    'expression': to_export,
    'fileFormat': 'PANDAS_DATAFRAME'
})

In [99]:
Map.addLayer(to_export)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [100]:
labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv')
fused_df = add_lucas_labels(fused_df, labels)
fused_df.drop(['geo', 'POINT_ID'], axis=1, inplace=True)
fused_df = fused_df.loc[fused_df['LABEL']!='NOT_CROP']

Loading files
Creating dataset of size 20705


100%|██████████| 20705/20705 [00:12<00:00, 1686.04it/s]


In [112]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'VH', 'VV']
band_df = []
labels = fused_df['LABEL']

for band in bands:
    df = fused_df.loc[:, [i for i in filter(lambda x: band in x, fused_df.columns)]]
    col_names = list(df.columns)
    col_names.sort(key=lambda x: int(x.split('_')[0]))
    df = df.reindex(col_names, axis=1)
    band_df.append(np.expand_dims(df.to_numpy(), 1))
    
